In [32]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

In [6]:
# read in from file
data = pd.read_csv("./Google-PlayStore.csv")

In [7]:
raw = data.copy()

In [8]:
# get numeric data description
data.describe().applymap("{0:.2f}".format)

,Rating,Rating Count,Minimum Installs,Maximum Installs,Price
count,2290061.00,2290061.00,2312837.00,2312944.00,2312944.00
mean,2.20,2864.84,183445.21,320201.71,0.10
std,2.11,212162.57,15131439.06,23554954.89,2.63
min,0.00,0.00,0.00,0.00,0.00
25%,0.00,0.00,50.00,84.00,0.00
50%,2.90,6.00,500.00,695.00,0.00
75%,4.30,42.00,5000.00,7354.00,0.00
max,5.00,138557570.00,10000000000.00,12057627016.00,400.00


In [9]:
# describe categorical data
data.describe(include=object)

,App Name,App Id,Category,Installs,Currency,Size,Minimum Android,Developer Id,Developer Website,Developer Email,Released,Last Updated,Content Rating,Privacy Policy,Scraped Time
count,2312942,2312944,2312944,2312837,2312809,2312748,2306414,2312911,1552109,2312913,2241891,2312944,2312944,1891991,2312944
unique,2177944,2312944,48,22,15,1657,154,758371,810440,950456,4158,3918,6,977743,67374
top,Tic Tac Toe,com.ishakwe.gakondo,Education,100+,USD,Varies with device,4.1 and up,Subsplash Inc,http://www.subsplash.com/,support@classplus.co,"Jun 16, 2020","Jun 10, 2021",Everyone,http://www.subsplash.com/legal/privacy,2021-06-16 07:35:57
freq,382,1,241090,443368,2311548,74777,604465,5422,7519,10345,2051,11186,2022089,7481,56


In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2312944 entries, 0 to 2312943
Data columns (total 24 columns):
 #   Column             Dtype  
---  ------             -----  
 0   App Name           object 
 1   App Id             object 
 2   Category           object 
 3   Rating             float64
 4   Rating Count       float64
 5   Installs           object 
 6   Minimum Installs   float64
 7   Maximum Installs   int64  
 8   Free               bool   
 9   Price              float64
 10  Currency           object 
 11  Size               object 
 12  Minimum Android    object 
 13  Developer Id       object 
 14  Developer Website  object 
 15  Developer Email    object 
 16  Released           object 
 17  Last Updated       object 
 18  Content Rating     object 
 19  Privacy Policy     object 
 20  Ad Supported       bool   
 21  In App Purchases   bool   
 22  Editors Choice     bool   
 23  Scraped Time       object 
dtypes: bool(4), float64(4), int64(1), object(15)
memor

In [11]:
# drop irrelavent features
data.drop("App Id", inplace=True, axis=1)
data.drop("Developer Id", inplace=True, axis=1)
data.drop("Developer Website", inplace=True, axis=1)
data.drop("Developer Email", inplace=True, axis=1)
data.drop("Privacy Policy", inplace=True, axis=1)
data.drop("Scraped Time", inplace=True, axis=1)
data.drop("Rating Count", inplace=True, axis=1)

In [12]:
# null value distribution
for c in data.columns:
    print(f'{c:20s} {data[c].isnull().sum(): 7d}')

App Name                   2
Category                   0
Rating                 22883
Installs                 107
Minimum Installs         107
Maximum Installs           0
Free                       0
Price                      0
Currency                 135
Size                     196
Minimum Android         6530
Released               71053
Last Updated               0
Content Rating             0
Ad Supported               0
In App Purchases           0
Editors Choice             0


In [13]:
# drop zero and null ratings to avoid imbalance
data = data[data['Rating'] != 0]
data.dropna(subset=['Rating'], inplace=True)

In [14]:
data.dropna(subset=['Released'], inplace=True)

In [15]:
data.dropna(subset=['Minimum Android'], inplace=True)

In [16]:
data.dropna(subset=['Currency'], inplace=True)

In [17]:
data.dropna(subset=['App Name'], inplace=True)

In [18]:
# null value distribution
for c in data.columns:
    print(f'{c:20s} {data[c].isnull().sum(): 7d}')

App Name                   0
Category                   0
Rating                     0
Installs                   0
Minimum Installs           0
Maximum Installs           0
Free                       0
Price                      0
Currency                   0
Size                       0
Minimum Android            0
Released                   0
Last Updated               0
Content Rating             0
Ad Supported               0
In App Purchases           0
Editors Choice             0


In [19]:
# only use max installs
data.drop("Installs", inplace=True, axis=1)
data.drop("Minimum Installs", inplace=True, axis=1)

In [20]:
print(data['Category'].value_counts())

Education                  126003
Tools                       82771
Entertainment               81955
Music & Audio               79316
Books & Reference           66272
Personalization             57546
Lifestyle                   53637
Business                    48045
Finance                     37073
Productivity                36221
Health & Fitness            32319
Puzzle                      31619
Shopping                    31271
Travel & Local              30652
Arcade                      30073
Casual                      28642
Sports                      25985
News & Magazines            25021
Social                      24709
Communication               23846
Photography                 22665
Food & Drink                22064
Simulation                  18869
Action                      17877
Adventure                   15496
Maps & Navigation           14159
Medical                     13627
Educational                 12201
Video Players & Editors     10434
Art & Design  

In [21]:
# transform size
data = data[data['Size'] != 'None']
data = data[data['Size'] != 'Varies with device']
data['Size'].str.strip().str[-1].unique()
def size(x):
    x = x.replace(',', '')
    if x[-1] == 'k':
        return float(x[:-1])
    elif x[-1] == 'M':
        return 1000 * float(x[:-1])
    else:
        return 1000000 * float(x[:-1])
data['Size'] = data['Size'].apply(size)

In [22]:
# transform Minimum Android
data = data[data['Minimum Android'] != 'Varies with device']

def sys(x):
    return float(x[:3])
data['Minimum Android'] = data['Minimum Android'].apply(sys)

In [23]:
# unify currency
data = data[data['Currency'] == 'USD']
data.drop("Currency", inplace=True, axis=1)

In [24]:
# extract Release Year and Month
data['Released'] = pd.to_datetime(data['Released'], format="%b %d, %Y")
data['Release Year'] = pd.DatetimeIndex(data['Released']).year
data['Release Month'] = pd.DatetimeIndex(data['Released']).month
data.drop('Released', inplace=True, axis=1)

In [25]:
# extract Update Year and Month
data['Last Updated'] = pd.to_datetime(data['Last Updated'], format="%b %d, %Y")
latest = data['Last Updated'].max()
data['Last Updated'] = (pd.to_datetime(latest) - pd.to_datetime(data["Last Updated"])).dt.days

In [26]:
# Extract App Name Length
def namelen(x):
    return int(len(x))
data['Name Length'] = data['App Name'].apply(namelen)
data.drop('App Name', inplace=True, axis=1)

In [27]:
data['Content Rating'].value_counts()

Everyone           1003687
Teen                102288
Mature 17+           34609
Everyone 10+         20252
Unrated                121
Adults only 18+         81
Name: Content Rating, dtype: int64

In [28]:
# transform boolean variables
data['Ad Supported'] = data['Ad Supported'].astype(int)
data['In App Purchases'] = data['In App Purchases'].astype(int)
data['Editors Choice'] = data['Editors Choice'].astype(int)
data['Free'] = data['Free'] = data['Free'].astype(int)

In [29]:
# drop categorical features
numerical = data.drop("Category", inplace=False, axis=1)
numerical.drop("Content Rating", inplace=True, axis=1)

In [30]:
numerical.head()

,Rating,Maximum Installs,Free,Price,Size,Minimum Android,Last Updated,Ad Supported,In App Purchases,Editors Choice,Release Year,Release Month,Name Length
1,4.4,7662,1,0.0,2900.0,5.0,41,1,0,0,2020,5,19
3,5.0,19,1,0.0,1800.0,4.0,977,1,0,0,2018,9,50
6,4.5,2567,1,0.0,2500.0,4.1,628,1,0,0,2019,9,32
7,2.0,702,1,0.0,16000.0,5.0,726,0,0,0,2019,6,21
9,4.7,62433,1,0.0,3500.0,4.1,252,1,0,0,2019,9,32


In [34]:
X = numerical.loc[:, ~numerical.columns.isin(['Rating'])]
y = numerical['Rating']
xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size=0.3, random_state=0)

In [35]:
xcol = X.columns
pd.DataFrame(xTrain, columns=xcol).to_csv('xTrain_num.csv', index=False)
pd.DataFrame(xTest, columns=xcol).to_csv('xTest_num.csv', index=False)
pd.DataFrame(yTrain, columns=['Rating']).to_csv('yTrain_num.csv', index=False)
pd.DataFrame(yTest, columns=['Rating']).to_csv('yTest_num.csv', index=False)

In [36]:
scaler = StandardScaler().fit(xTrain)
xTrain = scaler.transform(xTrain)
xTest = scaler.transform(xTest)

In [37]:
pd.DataFrame(xTrain, columns=xcol).to_csv('xTrain_num_std.csv', index=False)
pd.DataFrame(xTest, columns=xcol).to_csv('xTest_num_std.csv', index=False)

In [33]:
encoder =LabelEncoder()
data['Category']=encoder.fit_transform(data['Category'])

In [52]:
data.head()

,Category,Rating,Maximum Installs,Free,Price,Size,Minimum Android,Last Updated,Content Rating,Ad Supported,In App Purchases,Editors Choice,Release Year,Release Month,Name Length
1,42,4.4,7662,1,0.0,2900.0,5.0,41,Everyone,1,0,0,2020,5,19
3,13,5.0,19,1,0.0,1800.0,4.0,977,Everyone,1,0,0,2018,9,50
6,23,4.5,2567,1,0.0,2500.0,4.1,628,Everyone,1,0,0,2019,9,32
7,24,2.0,702,1,0.0,16000.0,5.0,726,Everyone,0,0,0,2019,6,21
9,31,4.7,62433,1,0.0,3500.0,4.1,252,Everyone,1,0,0,2019,9,32


In [53]:
ctf = ColumnTransformer([('encoder', OneHotEncoder(), ['Content Rating'])], remainder='passthrough')

dataset = ctf.fit_transform(data)

cols = ctf.get_feature_names()

cols[0] = '18+'
cols[1] = 'Everyone'
cols[2] = '10+'
cols[3] = '17+'
cols[4] = 'Teen'
cols[5] = 'Unrated'

dataset = pd.DataFrame(dataset, columns=cols)

C:\Users\brian\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [54]:
dataset.head()

,18+,Everyone,10+,17+,Teen,Unrated,Category,Rating,Maximum Installs,Free,Price,Size,Minimum Android,Last Updated,Ad Supported,In App Purchases,Editors Choice,Release Year,Release Month,Name Length
0,0.0,1.0,0.0,0.0,0.0,0.0,42.0,4.4,7662.0,1.0,0.0,2900.0,5.0,41.0,1.0,0.0,0.0,2020.0,5.0,19.0
1,0.0,1.0,0.0,0.0,0.0,0.0,13.0,5.0,19.0,1.0,0.0,1800.0,4.0,977.0,1.0,0.0,0.0,2018.0,9.0,50.0
2,0.0,1.0,0.0,0.0,0.0,0.0,23.0,4.5,2567.0,1.0,0.0,2500.0,4.1,628.0,1.0,0.0,0.0,2019.0,9.0,32.0
3,0.0,1.0,0.0,0.0,0.0,0.0,24.0,2.0,702.0,1.0,0.0,16000.0,5.0,726.0,0.0,0.0,0.0,2019.0,6.0,21.0
4,0.0,1.0,0.0,0.0,0.0,0.0,31.0,4.7,62433.0,1.0,0.0,3500.0,4.1,252.0,1.0,0.0,0.0,2019.0,9.0,32.0


In [59]:
X = dataset.loc[:, ~dataset.columns.isin(['Rating', 'Teen'])]
y = dataset['Rating']
xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size=0.3, random_state=0)

xcol = X.columns
pd.DataFrame(xTrain, columns=xcol).to_csv('xTrain_cat.csv', index=False)
pd.DataFrame(xTest, columns=xcol).to_csv('xTest_cat.csv', index=False)
pd.DataFrame(yTrain, columns=['Rating']).to_csv('yTrain_cat.csv', index=False)
pd.DataFrame(yTest, columns=['Rating']).to_csv('yTest_cat.csv', index=False)

scaler = StandardScaler()
xTrain = pd.DataFrame(xTrain, columns=xcol)
xTest = pd.DataFrame(xTest, columns=xcol)
xTrain[['Maximum Installs', 'Price', 'Size','Minimum Android', 'Last Updated', 'Release Year', 'Release Month', 'Name Length']] = scaler.fit_transform(xTrain[['Maximum Installs', 'Price', 'Size','Minimum Android', 'Last Updated', 'Release Year', 'Release Month', 'Name Length']])
xTest[['Maximum Installs', 'Price', 'Size','Minimum Android', 'Last Updated', 'Release Year', 'Release Month', 'Name Length']] = scaler.transform(xTest[['Maximum Installs', 'Price', 'Size','Minimum Android', 'Last Updated', 'Release Year', 'Release Month', 'Name Length']])
pd.DataFrame(xTrain, columns=xcol).to_csv('xTrain_cat_std.csv', index=False)
pd.DataFrame(xTest, columns=xcol).to_csv('xTest_cat_std.csv', index=False)